# NTB Mall Recommendation System using CBF

### Import library

In [1]:
import pandas as pd
import numpy as np
import math
import nltk
nltk.download("book")
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from bs4 import BeautifulSoup

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/mizanul/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /home/mizanul/nltk_data...
[nltk_data]    |   Package conll2002 is already up-to-date!
[nltk_data]    | Downloading package depend

## Preprocessing Dataset

### Load Dataset

In [2]:
df_base = pd.read_csv("./Clean_Dataset.csv")
df_base = pd.DataFrame(df_base)
df_base

,ID Produk,Tanggal Input,Nama Toko,Email Pelapak,Kategori Produk,Sub Kategori Produk,Nama Produk,Variasi,Harga,Stok,Deksripsi
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,2024-04-13 04:24:29,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Permen,Supermen,40 Gr,12000,0,<p>Permen susu khas lombok nusa tenggara barat...
1,066a5e68-7799-4f18-a49a-e2fefc556d1f,2024-04-13 04:24:30,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Permen,Supermen,60 Gr,16000,0,<p>Permen susu khas lombok nusa tenggara barat...
2,b70def04-da59-4c2e-912a-e5944579831d,2023-10-29 07:45:51,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Coklat,Semeton Choco (Coklat Lombok),NaN,15000,37,<p>Coklat susu adalah mahakarya gurih yang mem...
3,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,2023-10-19 09:45:43,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Lanang,NaN,19000,20,<p>Kopi Lanang Bubuk Ground&nbsp;<br>Level Gri...
4,9271605a-009d-4364-8deb-f8d04847e38b,2023-10-19 09:35:33,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Lombok Murni,NaN,28000,20,<p>Kopi denga kemasaran Pouch <br>100% Tanpa C...
...,...,...,...,...,...,...,...,...,...,...,...
1911,bbfe46af-ca0a-4335-a329-67925c72d4fd,2021-03-25 13:42:44,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Madu,Mbojo Honey,NaN,169000,0,<p>Mbojo Honey<br></p>
1912,f2d75215-debe-4e1c-bcea-a3aaa468b50c,2021-03-25 13:38:11,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Bubuk 100 Gr,NaN,29000,0,<p>Kopi Bubuk Pesanggerahan 100 Gr</p><p><span...
1913,cb760741-76be-4c4b-99ee-0b4c671f0c8a,2021-03-25 13:34:01,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Makanan & Minuman,Kopi,Kopi Jahe 100 Gr,NaN,29000,0,<p>Kopi Jahe Pesanggerahan 100 Gr</p><p><span ...
1914,3b90f8a5-bbef-466f-94a8-ce275b3fee70,2021-03-25 13:32:03,NTB MALL OFFICIAL,ntbmall2020@gmail.com,Fashion & Bag,Fashion Wanita,Dompet Motif Krealang Sumbawa,NaN,455000,5,<p>Dompet Motif Krealang Sumbawa<br></p>


In [3]:
df_preprocess  = df_base.drop(["Tanggal Input", "Nama Toko", "Email Pelapak", "Variasi", "Stok"], axis=1)
df_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID Produk            1916 non-null   object
 1   Kategori Produk      1916 non-null   object
 2   Sub Kategori Produk  1916 non-null   object
 3   Nama Produk          1916 non-null   object
 4   Harga                1916 non-null   int64 
 5   Deksripsi            1916 non-null   object
dtypes: int64(1), object(5)
memory usage: 89.9+ KB


### Check Missing Values and Data Duplications

In [4]:
df_preprocess.isnull().sum()

ID Produk              0
Kategori Produk        0
Sub Kategori Produk    0
Nama Produk            0
Harga                  0
Deksripsi              0
dtype: int64

In [5]:
df_preprocess['Kategori Produk'] = df_preprocess['Kategori Produk'].astype(str)
df_preprocess['Nama Produk'] = df_preprocess['Nama Produk'].astype(str)
df_preprocess['Harga'] = df_preprocess['Harga'].astype(str)
df_preprocess['Deksripsi'] = df_preprocess['Deksripsi'].astype(str)
df_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID Produk            1916 non-null   object
 1   Kategori Produk      1916 non-null   object
 2   Sub Kategori Produk  1916 non-null   object
 3   Nama Produk          1916 non-null   object
 4   Harga                1916 non-null   object
 5   Deksripsi            1916 non-null   object
dtypes: object(6)
memory usage: 89.9+ KB


In [6]:
df_preprocess.duplicated().sum()

41

The dataset contain 41 data duplications

In [7]:
duplicate_rows = df_preprocess[df_preprocess.duplicated(subset=['ID Produk'], keep=False)]
print(duplicate_rows)

                                 ID Produk    Kategori Produk  \
0     066a5e68-7799-4f18-a49a-e2fefc556d1f  Makanan & Minuman   
1     066a5e68-7799-4f18-a49a-e2fefc556d1f  Makanan & Minuman   
118   56a0c7d5-cdc3-4e24-a189-e341848fe33f      Fashion & Bag   
119   56a0c7d5-cdc3-4e24-a189-e341848fe33f      Fashion & Bag   
120   56a0c7d5-cdc3-4e24-a189-e341848fe33f      Fashion & Bag   
...                                    ...                ...   
1862  17a50ed6-538c-481a-946c-f8e68d3c2069      Fashion & Bag   
1864  e1f4b3e0-991b-4d60-b019-6efc4141214f      Fashion & Bag   
1865  e1f4b3e0-991b-4d60-b019-6efc4141214f      Fashion & Bag   
1873  cc1f8f30-586e-4d2e-a12f-fd47483a1bb4      Fashion & Bag   
1874  cc1f8f30-586e-4d2e-a12f-fd47483a1bb4      Fashion & Bag   

     Sub Kategori Produk                                Nama Produk   Harga  \
0                 Permen                                   Supermen   12000   
1                 Permen                                   Su

In [8]:
df_preprocess = df_preprocess.drop_duplicates(subset=['ID Produk'], keep='first').reset_index(drop=True)

df_preprocess

,ID Produk,Kategori Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Makanan & Minuman,Permen,Supermen,12000,<p>Permen susu khas lombok nusa tenggara barat...
1,b70def04-da59-4c2e-912a-e5944579831d,Makanan & Minuman,Coklat,Semeton Choco (Coklat Lombok),15000,<p>Coklat susu adalah mahakarya gurih yang mem...
2,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Makanan & Minuman,Kopi,Kopi Lanang,19000,<p>Kopi Lanang Bubuk Ground&nbsp;<br>Level Gri...
3,9271605a-009d-4364-8deb-f8d04847e38b,Makanan & Minuman,Kopi,Kopi Lombok Murni,28000,<p>Kopi denga kemasaran Pouch <br>100% Tanpa C...
4,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion & Bag,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,<p>Lestary Tenun Batik sasambo nlouse Kuning&n...
...,...,...,...,...,...,...
1853,bbfe46af-ca0a-4335-a329-67925c72d4fd,Makanan & Minuman,Madu,Mbojo Honey,169000,<p>Mbojo Honey<br></p>
1854,f2d75215-debe-4e1c-bcea-a3aaa468b50c,Makanan & Minuman,Kopi,Kopi Bubuk 100 Gr,29000,<p>Kopi Bubuk Pesanggerahan 100 Gr</p><p><span...
1855,cb760741-76be-4c4b-99ee-0b4c671f0c8a,Makanan & Minuman,Kopi,Kopi Jahe 100 Gr,29000,<p>Kopi Jahe Pesanggerahan 100 Gr</p><p><span ...
1856,3b90f8a5-bbef-466f-94a8-ce275b3fee70,Fashion & Bag,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,<p>Dompet Motif Krealang Sumbawa<br></p>


In [9]:
df_preprocess.duplicated().sum()

0

### Remove HTML Tag on Deksripsi column

In [10]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    text_without_tags = soup.get_text()
    return ' '.join(text_without_tags.split())


df_preprocess['Deksripsi'] = df_preprocess['Deksripsi'].apply(remove_html_tags)
df_preprocess

,ID Produk,Kategori Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Makanan & Minuman,Permen,Supermen,12000,Permen susu khas lombok nusa tenggara barat ya...
1,b70def04-da59-4c2e-912a-e5944579831d,Makanan & Minuman,Coklat,Semeton Choco (Coklat Lombok),15000,Coklat susu adalah mahakarya gurih yang memika...
2,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Makanan & Minuman,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...
3,9271605a-009d-4364-8deb-f8d04847e38b,Makanan & Minuman,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...
4,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion & Bag,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,Lestary Tenun Batik sasambo nlouse Kuning SIZE M
...,...,...,...,...,...,...
1853,bbfe46af-ca0a-4335-a329-67925c72d4fd,Makanan & Minuman,Madu,Mbojo Honey,169000,Mbojo Honey
1854,f2d75215-debe-4e1c-bcea-a3aaa468b50c,Makanan & Minuman,Kopi,Kopi Bubuk 100 Gr,29000,Kopi Bubuk Pesanggerahan 100 Gr#kopi #kopilomb...
1855,cb760741-76be-4c4b-99ee-0b4c671f0c8a,Makanan & Minuman,Kopi,Kopi Jahe 100 Gr,29000,Kopi Jahe Pesanggerahan 100 Gr#kopi #kopilombo...
1856,3b90f8a5-bbef-466f-94a8-ce275b3fee70,Fashion & Bag,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,Dompet Motif Krealang Sumbawa


### Create Concantenated column and Case Foldings Step

Feature yang digunakan dalam column Concatenated adalah gabungan dari column: 

Concantenated = (Nama Produk, Harga dan Deskripsi)

In [11]:
df_preprocess['Concatenated'] = df_preprocess.apply(lambda row: ' '.join(row[2:]), axis=1)
df_preprocess

,ID Produk,Kategori Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi,Concatenated
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Makanan & Minuman,Permen,Supermen,12000,Permen susu khas lombok nusa tenggara barat ya...,Permen Supermen 12000 Permen susu khas lombok ...
1,b70def04-da59-4c2e-912a-e5944579831d,Makanan & Minuman,Coklat,Semeton Choco (Coklat Lombok),15000,Coklat susu adalah mahakarya gurih yang memika...,Coklat Semeton Choco (Coklat Lombok) 15000 Cok...
2,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Makanan & Minuman,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...,Kopi Kopi Lanang 19000 Kopi Lanang Bubuk Groun...
3,9271605a-009d-4364-8deb-f8d04847e38b,Makanan & Minuman,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...,Kopi Kopi Lombok Murni 28000 Kopi denga kemasa...
4,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion & Bag,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,Lestary Tenun Batik sasambo nlouse Kuning SIZE M,Fashion Wanita Lestary Tenun Batik Sasambo Blo...
...,...,...,...,...,...,...,...
1853,bbfe46af-ca0a-4335-a329-67925c72d4fd,Makanan & Minuman,Madu,Mbojo Honey,169000,Mbojo Honey,Madu Mbojo Honey 169000 Mbojo Honey
1854,f2d75215-debe-4e1c-bcea-a3aaa468b50c,Makanan & Minuman,Kopi,Kopi Bubuk 100 Gr,29000,Kopi Bubuk Pesanggerahan 100 Gr#kopi #kopilomb...,Kopi Kopi Bubuk 100 Gr 29000 Kopi Bubuk Pesang...
1855,cb760741-76be-4c4b-99ee-0b4c671f0c8a,Makanan & Minuman,Kopi,Kopi Jahe 100 Gr,29000,Kopi Jahe Pesanggerahan 100 Gr#kopi #kopilombo...,Kopi Kopi Jahe 100 Gr 29000 Kopi Jahe Pesangge...
1856,3b90f8a5-bbef-466f-94a8-ce275b3fee70,Fashion & Bag,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,Dompet Motif Krealang Sumbawa,Fashion Wanita Dompet Motif Krealang Sumbawa 4...


In [12]:
df_preprocess['Concatenated'] = df_preprocess['Concatenated'].str.lower()
df_preprocess


,ID Produk,Kategori Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi,Concatenated
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Makanan & Minuman,Permen,Supermen,12000,Permen susu khas lombok nusa tenggara barat ya...,permen supermen 12000 permen susu khas lombok ...
1,b70def04-da59-4c2e-912a-e5944579831d,Makanan & Minuman,Coklat,Semeton Choco (Coklat Lombok),15000,Coklat susu adalah mahakarya gurih yang memika...,coklat semeton choco (coklat lombok) 15000 cok...
2,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Makanan & Minuman,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...,kopi kopi lanang 19000 kopi lanang bubuk groun...
3,9271605a-009d-4364-8deb-f8d04847e38b,Makanan & Minuman,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...,kopi kopi lombok murni 28000 kopi denga kemasa...
4,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion & Bag,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,Lestary Tenun Batik sasambo nlouse Kuning SIZE M,fashion wanita lestary tenun batik sasambo blo...
...,...,...,...,...,...,...,...
1853,bbfe46af-ca0a-4335-a329-67925c72d4fd,Makanan & Minuman,Madu,Mbojo Honey,169000,Mbojo Honey,madu mbojo honey 169000 mbojo honey
1854,f2d75215-debe-4e1c-bcea-a3aaa468b50c,Makanan & Minuman,Kopi,Kopi Bubuk 100 Gr,29000,Kopi Bubuk Pesanggerahan 100 Gr#kopi #kopilomb...,kopi kopi bubuk 100 gr 29000 kopi bubuk pesang...
1855,cb760741-76be-4c4b-99ee-0b4c671f0c8a,Makanan & Minuman,Kopi,Kopi Jahe 100 Gr,29000,Kopi Jahe Pesanggerahan 100 Gr#kopi #kopilombo...,kopi kopi jahe 100 gr 29000 kopi jahe pesangge...
1856,3b90f8a5-bbef-466f-94a8-ce275b3fee70,Fashion & Bag,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,Dompet Motif Krealang Sumbawa,fashion wanita dompet motif krealang sumbawa 4...


### Main Data Preprocessing

In [13]:
def ganti_isi_column(data_lama, data_baru):
  for i in range(len(data_lama)):
    data_lama[i] = data_baru[i]
  return data_lama

In [14]:
import re

def preprocessing(document_to_be_scored):
  # Tokenize Dataset
    pisah = {}
    for i in range(len(document_to_be_scored)):
        text = document_to_be_scored.values[i]
        # Remove punctuation using regular expressions
        text = re.sub(r'[.,!?"\'()@#%^&*]', ' ', text)
        text = re.sub(r'[-:]', ' ', text)

        pisah[i] = text.split()

    # Menghilangkan stopword menggunakan library nltk
    stopword = ['dengan']  # custom list stopword

    removed = {}
    hasil = []
    listStopword = stopwords.words('indonesian')  # list stopword dari NLTK

    # Tambahkan stopword
    for i in stopword:
        listStopword.append(i)

    # listStopword = set(stopwords.words('indonesian')) #list stopword
    hitung = 0  # untuk menghitung frekuensi
    for i in pisah.values():
        for j in i:
            if j not in listStopword:  # jika setiap kata tidak ada di list stopword
                hasil.append(j)
        removed[hitung] = hasil[:]
        del hasil[:]
        hitung = hitung + 1

    # Memakai library sastrawi untuk melakukan stemming
    factory = StemmerFactory()  # membuat objek stemmer
    stemmer = factory.create_stemmer()
    hitung = 0

    hasil = []
    hasil2 = {}

    for i in removed.values():
        for j in i:
            # Hasil Stemming
            hasil.append(stemmer.stem(j))
        hasil2[hitung] = hasil[:]
        del hasil[:]
        hitung = hitung + 1
    hasil = ""
    korpus2 = []
    for i in hasil2.values():
        hasil = ""
        for j in i:
            hasil = hasil + " " + j
        korpus2.append(hasil)

    # Timpa hasil
    document_to_be_scored = ganti_isi_column(document_to_be_scored, korpus2)

    return document_to_be_scored

8 minutes run

In [15]:
df_preprocess['new_feature'] = preprocessing(df_preprocess['Concatenated'])

In [16]:
df_preprocess.to_csv('clean_preprocessing.csv', index=False)

In [ ]:
df_preprocess[][1855]

,ID Produk,Kategori Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi,Concatenated,new_feature
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Makanan & Minuman,Permen,Supermen,12000,Permen susu khas lombok nusa tenggara barat ya...,permen supermen 12000 permen susu khas lombok...,permen supermen 12000 permen susu khas lombok...
1,b70def04-da59-4c2e-912a-e5944579831d,Makanan & Minuman,Coklat,Semeton Choco (Coklat Lombok),15000,Coklat susu adalah mahakarya gurih yang memika...,coklat semeton choco coklat lombok 15000 cokl...,coklat semeton choco coklat lombok 15000 cokl...
2,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Makanan & Minuman,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...,kopi kopi lanang 19000 kopi lanang bubuk grou...,kopi kopi lanang 19000 kopi lanang bubuk grou...
3,9271605a-009d-4364-8deb-f8d04847e38b,Makanan & Minuman,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...,kopi kopi lombok murni 28000 kopi denga kemas...,kopi kopi lombok murni 28000 kopi denga kemas...
4,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion & Bag,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,Lestary Tenun Batik sasambo nlouse Kuning SIZE M,fashion wanita lestary tenun batik sasambo bl...,fashion wanita lestary tenun batik sasambo bl...
...,...,...,...,...,...,...,...,...
1853,bbfe46af-ca0a-4335-a329-67925c72d4fd,Makanan & Minuman,Madu,Mbojo Honey,169000,Mbojo Honey,madu mbojo honey 169000 mbojo honey,madu mbojo honey 169000 mbojo honey
1854,f2d75215-debe-4e1c-bcea-a3aaa468b50c,Makanan & Minuman,Kopi,Kopi Bubuk 100 Gr,29000,Kopi Bubuk Pesanggerahan 100 Gr#kopi #kopilomb...,kopi kopi bubuk 100 gr 29000 kopi bubuk sangg...,kopi kopi bubuk 100 gr 29000 kopi bubuk sangg...
1855,cb760741-76be-4c4b-99ee-0b4c671f0c8a,Makanan & Minuman,Kopi,Kopi Jahe 100 Gr,29000,Kopi Jahe Pesanggerahan 100 Gr#kopi #kopilombo...,kopi kopi jahe 100 gr 29000 kopi jahe sangger...,kopi kopi jahe 100 gr 29000 kopi jahe sangger...
1856,3b90f8a5-bbef-466f-94a8-ce275b3fee70,Fashion & Bag,Fashion Wanita,Dompet Motif Krealang Sumbawa,455000,Dompet Motif Krealang Sumbawa,fashion wanita dompet motif krealang sumbawa ...,fashion wanita dompet motif krealang sumbawa ...


## Content Base Filtering Algorithm

In [2]:
df_clean = pd.DataFrame(pd.read_csv('clean_preprocessing.csv'))
df_clean.head()

,ID Produk,Kategori Produk,Sub Kategori Produk,Nama Produk,Harga,Deksripsi,Concatenated,new_feature
0,066a5e68-7799-4f18-a49a-e2fefc556d1f,Makanan & Minuman,Permen,Supermen,12000,Permen susu khas lombok nusa tenggara barat ya...,permen supermen 12000 permen susu khas lombok...,permen supermen 12000 permen susu khas lombok...
1,b70def04-da59-4c2e-912a-e5944579831d,Makanan & Minuman,Coklat,Semeton Choco (Coklat Lombok),15000,Coklat susu adalah mahakarya gurih yang memika...,coklat semeton choco coklat lombok 15000 cokl...,coklat semeton choco coklat lombok 15000 cokl...
2,48e9e810-6bd9-4cd2-b73d-84b380e17c9d,Makanan & Minuman,Kopi,Kopi Lanang,19000,Kopi Lanang Bubuk Ground Level Grinder Coffee ...,kopi kopi lanang 19000 kopi lanang bubuk grou...,kopi kopi lanang 19000 kopi lanang bubuk grou...
3,9271605a-009d-4364-8deb-f8d04847e38b,Makanan & Minuman,Kopi,Kopi Lombok Murni,28000,Kopi denga kemasaran Pouch 100% Tanpa Campuran...,kopi kopi lombok murni 28000 kopi denga kemas...,kopi kopi lombok murni 28000 kopi denga kemas...
4,f8cea1e9-929f-4138-a96a-4c73bcfa620e,Fashion & Bag,Fashion Wanita,Lestary Tenun Batik Sasambo Blouse Kuning,610000,Lestary Tenun Batik sasambo nlouse Kuning SIZE M,fashion wanita lestary tenun batik sasambo bl...,fashion wanita lestary tenun batik sasambo bl...


### Compute TF-IDF Features

In [3]:
def compute_tfidf(documents):
    # Step 1: Tokenization
    def tokenize(document):
        return document.lower().split()

    # Step 2: Create a vocabulary
    vocabulary = set()
    for doc in documents:
        vocabulary.update(tokenize(doc))

    vocabulary = sorted(vocabulary)

    # Step 3: Compute TF (Term Frequency)
    print("\nIterasi pada setiap data:")
    count=1
    tf_matrix = []
    for doc in documents:
        tokens = tokenize(doc)
        tf = [tokens.count(word) for word in vocabulary]
        print("Data Ke-",count,":",tokens)
        tf_matrix.append(tf)
        count+=1


    # Step 4: Compute IDF (Inverse Document Frequency)
    idf = []
    N = len(documents)
    for word in vocabulary:
        doc_count = sum([1 for doc in documents if word in tokenize(doc)])
        idf.append(math.log(N / (1 + doc_count)))


    # Step 5: Compute TF-IDF
    tfidf_matrix = []
    for tf_vector in tf_matrix:
        tfidf = [tf * idf_value for tf, idf_value in zip(tf_vector, idf)]
        tfidf_matrix.append(tfidf)
    
    return vocabulary, tf_matrix, idf, tfidf_matrix

In [33]:
documents = df_clean['new_feature']

vocabulary,tf_matrix,idf,tfidf_matrix = compute_tfidf(documents)


Iterasi pada setiap data:
Data Ke- 1 : ['permen', 'supermen', '12000', 'permen', 'susu', 'khas', 'lombok', 'nusa', 'tenggara', 'barat', 'sehat', 'kandung', 'gula', 'rusak', 'citra', 'susu', 'permen']
Data Ke- 2 : ['coklat', 'semeton', 'choco', 'coklat', 'lombok', '15000', 'coklat', 'susu', 'mahakarya', 'gurih', 'mikat', 'indera', 'kali', 'mata', 'temu', 'dengan', 'warna', 'coklat', 'goda', 'pancar', 'aroma', 'manis', 'rayu', 'perut', 'jiwa', 'kali', 'gigit', 'sentuh', 'bibir', 'coklat', 'susu', 'lembut', 'belai', 'cair', 'mulut', 'lezat', 'lupa', 'padu', 'sempurna', 'manis', 'gurih', 'tari', 'harmoni', 'manja', 'lidah', 'coklat', 'susu', 'goda', 'sulit', 'tolak', 'undang', 'rasa', 'coklat', 'ntb', 'coklatntb']
Data Ke- 3 : ['kopi', 'kopi', 'lanang', '19000', 'kopi', 'lanang', 'bubuk', 'ground', 'level', 'grinder', 'coffee', 'fine', 'halus', 'roasting', 'medium', 'to', 'darknetto', '100', 'g']
Data Ke- 4 : ['kopi', 'kopi', 'lombok', 'murni', '28000', 'kopi', 'denga', 'kemasaran', 'pouc

Save model to csv

In [34]:
import csv

csv_file_path = 'tfidf_features.csv'

# Save the TF-IDF matrix to a CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header row (vocabulary)
    writer.writerow(vocabulary)

    # Write each TF-IDF vector as a row
    for tfidf_vector in tfidf_matrix:
        writer.writerow(tfidf_vector)

print(f'TF-IDF features saved to {csv_file_path}')

TF-IDF features saved to tfidf_features.csv


### Unique Feature in Sub Kategori Produk

In [4]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID Produk            1858 non-null   object
 1   Kategori Produk      1858 non-null   object
 2   Sub Kategori Produk  1858 non-null   object
 3   Nama Produk          1858 non-null   object
 4   Harga                1858 non-null   int64 
 5   Deksripsi            1858 non-null   object
 6   Concatenated         1858 non-null   object
 7   new_feature          1858 non-null   object
dtypes: int64(1), object(7)
memory usage: 116.2+ KB


In [5]:
# Count the occurrences of each unique value in the 'Category' column
value_counts = df_clean['Kategori Produk'].value_counts()

pd.set_option('display.max_rows', None)

print("Value counts in 'Category' column:")
print(value_counts)

Value counts in 'Category' column:
Kategori Produk
Makanan & Minuman      558
Fashion & Bag          501
Mutiara & Aksesoris    239
Kerajinan              202
Kain Tenun             122
Kain Songket           114
Kecantikan              55
Merchandise             25
Kesehatan               18
Bela Pengadaan          11
Lainnya                  8
Sembako                  5
Name: count, dtype: int64


### Euclidean Distance

In [66]:
def euclidean_distance(data, limit, index,detail_output=False):

    # Baca file CSV yang berisi fitur TF-IDF
    df_tfidf = pd.read_csv('tfidf_features.csv')

    # Pilih kolom yang berisi vektor TF-IDF, misalnya 'tfidf_vector'
    tfidf_matrix = tfidf_matrix = df_tfidf.iloc[::] 

    # Fungsi untuk menghitung Euclidean Distance antara dua vektor
    def euclidean_distance(vector1, vector2):
        return np.linalg.norm(vector1 - vector2)

    # Hitung kemiripan dokumen 1 dengan dokumen lainnya
    document_1 = tfidf_matrix.iloc[index]  # Ganti dengan indeks dokumen yang ingin Anda bandingkan
    label_document_1 = df_clean['Kategori Produk'][index]

    if(detail_output==True):
        print("=====Detail Query Acuan: ======")

        print(label_document_1)
        print(df_clean["ID Produk"][index])
        print(df_clean["Sub Kategori Produk"][index])
        print(df_clean["Nama Produk"][index])
        print(df_clean["Harga"][index])
        print(df_clean["Deksripsi"][index])
        print('\n\n')


    #digunakan untuk menyimpan hasil perhitungan jarak Euclidean antara dokumen referensi dan dokumen-dokumen lainnya.
    similarities = []

    for i in range(1, len(tfidf_matrix)):
        document_i = tfidf_matrix.iloc[i]
        distance = euclidean_distance(document_1, document_i)
        similarities.append((i, distance))

    # Sort hasil kemiripan berdasarkan jarak (semakin kecil semakin mirip)
    similarities.sort(key=lambda x: x[1])

    # Remove first index
    similarities = similarities[1:]


    count = 0

    # Label dokumen referensi
    label_reference = label_document_1

    # Menginisialisasi list label sebenarnya dan prediksi
    true_labels = []
    predicted_labels = []
    for i, distance in similarities:
        if(detail_output==True):
            print(f"Dokumen {i}: \nEuclidean Distance = {distance}")
            print("ID Document              : ",df_clean['ID Produk'][i])
            print("Kategori Produk          : ",df_clean['Kategori Produk'][i])
            print("Sub-Kategori Produk      : ",df_clean['Sub Kategori Produk'][i])
            print("Nama Produk              : ",df_clean['Nama Produk'][i])
            print("Concate Description      : ",df_clean['Concatenated'][i])
            print("\n")
        
        else:
            data['True_Label'].append(label_document_1)
            data['Predicted_label'].append(df_clean['Kategori Produk'][i])
            data['Predicted Sub Category'].append(df_clean['Sub Kategori Produk'][i])
            data['Nama Produk'].append(df_clean['Nama Produk'][i])
            data['Deskripsi'].append(df_clean['Deksripsi'][i])
            data['ID Produk'].append(df_clean['ID Produk'][i])
            data['Similarity'].append(distance)

        if ((count+1)==limit): break
        else: count+=1

        true_label = df_clean['Kategori Produk'][i]
        true_labels.append(label_document_1)

        # print(label_document_1,"====",true_label)
        
        if label_document_1 == true_label:
            # Dokumen referensi memiliki jarak Euclidean 0, itu adalah dirinya sendiri
            predicted_labels.append(label_reference)

        else:
            # Dokumen dengan jarak Euclidean terkecil dianggap sebagai prediksi
            predicted_labels.append(true_label)


    from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score

    # Generate classification report
    index_mrr = next((i for i, label in enumerate(predicted_labels) if label == label_reference), -1) + 1

    accuracy_scr = accuracy_score(true_labels, predicted_labels)
    if(index_mrr == 0):
        mrr = 0
    else:
        mrr = 1/index_mrr


    if(detail_output==True):
        confusion_mat = confusion_matrix(true_labels,predicted_labels)
        classification_rep = classification_report(true_labels, predicted_labels)

        print("Classification Report:")
        print(confusion_mat)
        print(classification_rep)

    print(label_document_1," Acc ==> ",accuracy_scr)
    print("index_mrr :===> ",index_mrr)
    print("mrr score :===> ",mrr)



    return accuracy_scr, mrr

### Cosine Similarity

In [71]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_limit(data,limit, index, detail_output=False):
    # Load the TF-IDF feature matrix (assuming it's already computed)
    df_tfidf = pd.read_csv('tfidf_features.csv')

    # Select the column containing the TF-IDF vectors
    tfidf_matrix = df_tfidf.iloc[::]

    # Calculate the cosine similarity between the reference document and other documents
    reference_vector = tfidf_matrix.iloc[index]  # Reference document
    similarities = cosine_similarity([reference_vector], tfidf_matrix)
    label_document_1 = df_clean['Kategori Produk'][index]
    
    if(detail_output==True):
        print("=====Detail Query Acuan: ======")

        print(label_document_1)
        print(df_clean["ID Produk"][index])
        print(df_clean["Sub Kategori Produk"][index])
        print(df_clean["Nama Produk"][index])
        print(df_clean["Harga"][index])
        print(df_clean["Deksripsi"][index])
        print('\n\n')

    # Sort the similarities
    similar_indices = (-similarities).argsort()[0][1:limit+1]  # Sort in descending order

    # Initialize lists for true labels and predicted labels
    true_labels = []
    predicted_labels = []

    count=0
    for i in similar_indices:  # Excluding the reference document
        similar = similarities[0][i]
        if(detail_output==True):
            print(f"Dokumen {i}: \nEuclidean Distance = {similar}")
            print("ID Document              : ",df_clean['ID Produk'][i])
            print("Kategori Produk          : ",df_clean['Kategori Produk'][i])
            print("Sub-Kategori Produk      : ",df_clean['Sub Kategori Produk'][i])
            print("Nama Produk              : ",df_clean['Nama Produk'][i])
            print("Concate Description      : ",df_clean['Concatenated'][i])
            print("\n")
        
        else:
            data['True_Label'].append(label_document_1)
            data['Predicted_label'].append(df_clean['Kategori Produk'][i])
            data['Predicted Sub Category'].append(df_clean['Sub Kategori Produk'][i])
            data['Nama Produk'].append(df_clean['Nama Produk'][i])
            data['Deskripsi'].append(df_clean['Deksripsi'][i])
            data['ID Produk'].append(df_clean['ID Produk'][i])
            data['Similarity'].append(similar)

        if (count==limit): break
        else: count+=1

        true_label = df_clean['Kategori Produk'][i]
        true_labels.append(label_document_1)

        # print(label_document_1,"====",true_label)
        
        if label_document_1 == true_label:
            # Dokumen referensi memiliki jarak Euclidean 0, itu adalah dirinya sendiri
            predicted_labels.append(label_document_1)
        else:
            # Dokumen dengan jarak Euclidean terkecil dianggap sebagai prediksi
            predicted_labels.append(true_label)

    from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

    # Generate classification report
    index_mrr = next((i for i, label in enumerate(predicted_labels) if label == label_document_1), -1) + 1

    accuracy_scr = accuracy_score(true_labels, predicted_labels)
    if(index_mrr == 0):
        mrr = 0
    else:
        mrr = 1/index_mrr

    if(detail_output==True):
        confusion_mat = confusion_matrix(true_labels,predicted_labels)
        classification_rep = classification_report(true_labels, predicted_labels)

        print("Classification Report:")
        print(confusion_mat)
        print(classification_rep)

    print(label_document_1," ==> ",accuracy_scr)
    print("index_mrr :===> ",index_mrr)
    print("mrr score :===> ",mrr)

    return accuracy_scr, mrr

## Testing

### One sample testing

In [67]:
# Tampilkan dokumen yang paling mirip dengan dokumen pertama

data = {
    'True_Label': [],
    'Predicted_label': [],
    'Predicted Sub Category': [],
    'Nama Produk':[],
    'ID Produk': [],
    'Similarity': [],
    'Deskripsi':[],
}

limit = 5
n_training = 1
acc=0
prc=0
recall=0
index_document = 25

for i in range(n_training):
    euclidean_accuracy, euclidean_mrr = euclidean_distance(data,limit,index_document,detail_output=True)
    acc +=euclidean_accuracy
    print('================================================================================================')


print("Hasil Evaluasi Akhir: ",acc/n_training)

=====Detail Query Acuan: ======
Kain Songket
4252c2ba-741c-489e-869f-dbed95f053c8
Kain Songket Lombok
Kain Tenun Motif Matahari
875000
Kain Tenun Motif Matahari



Dokumen 833: 
Euclidean Distance = 12.492796268703362
ID Document              :  6798a296-06fe-4769-a619-a24a1b0069eb
Kategori Produk          :  Kain Tenun
Sub-Kategori Produk      :  Kain Tenun Lombok
Nama Produk              :  Kain Motif Matahari
Concate Description      :   kain tenun lombok kain motif matahari 936000 kain motif mataharibeli ntb mall ntb mall semuaadasemuabisa thebestchoiceoflocalbrand produklokal rajin bambu ntbmall oleholehlombok khaslombok lombok ntb sumbawa beladanbeliproduklokal banggabuatanindonesia kaintenun songket makanankhas jalanjalan tempatwisata marketplacentb


Dokumen 850: 
Euclidean Distance = 12.492796268703362
ID Document              :  720f9041-a5a1-4963-9adb-fdc0d4b8c514
Kategori Produk          :  Kain Tenun
Sub-Kategori Produk      :  Kain Tenun Lombok
Nama Produk              : 

/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [73]:
# Tampilkan dokumen yang paling mirip dengan dokumen pertama
data = {
    'True_Label': [],
    'Predicted_label': [],
    'Predicted Sub Category': [],
    'Nama Produk':[],
    'ID Produk': [],
    'Similarity': [],
    'Deskripsi':[],
}

limit = 5
n_training = 1
acc=0
index_document = 25

for i in range(n_training):
    cosine, cosine_mrr = cosine_similarity_limit(data,limit,index_document,detail_output=True)
    acc+=cosine

print(acc/n_training)
print(cosine_mrr/n_training)

=====Detail Query Acuan: ======
Kain Songket
4252c2ba-741c-489e-869f-dbed95f053c8
Kain Songket Lombok
Kain Tenun Motif Matahari
875000
Kain Tenun Motif Matahari



Dokumen 511: 
Euclidean Distance = 0.5782018523701766
ID Document              :  e76209cb-298a-4ec2-a127-615812114ccd
Kategori Produk          :  Kain Tenun
Sub-Kategori Produk      :  Kain Tenun Lombok
Nama Produk              :  Kain Motif Matahari
Concate Description      :   kain tenun lombok kain motif matahari 936000 kain motif matahari sr pakai adat hari upacara adat giat agama beli ntb mall ntb mall semuaadasemuabisa thebestchoiceoflocalbrand produklokal rajin bambu ntbmall oleholehlombok khaslombok lombok ntb sumbawa beladanbeliproduklokal banggabuatanindonesia kaintenun songket makanankhas jalanjalan tempatwisata marketplacentb


Dokumen 833: 
Euclidean Distance = 0.5748955771033203
ID Document              :  6798a296-06fe-4769-a619-a24a1b0069eb
Kategori Produk          :  Kain Tenun
Sub-Kategori Produk      :  K

/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mizanul/Documents/code/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Multiple Testing

In [68]:
# Tampilkan dokumen yang paling mirip dengan dokumen pertama

data = {
    'True_Label': [],
    'Predicted_label': [],
    'Predicted Sub Category': [],
    'Nama Produk':[],
    'ID Produk': [],
    'Similarity': [],
    'Deskripsi':[],
}

limit = 15
n_training = 1500
acc=0
mrr=0

for i in range(n_training):
    index_document = i
    euclidean, euclidean_mrr = euclidean_distance(data,limit,index_document,detail_output=False)
    acc+=euclidean
    mrr += euclidean_mrr

result_df = pd.DataFrame(data)

# Menyimpan DataFrame ke dalam file CSV
result_df.to_csv('euqlidean_output.csv', index=False)

print("Mean Accuracy: ", acc/n_training)
print("MRR Score: ", mrr/n_training)


Makanan & Minuman  Acc ==>  0.2857142857142857
index_mrr :===>  1
mrr score :===>  1.0
Makanan & Minuman  Acc ==>  0.5714285714285714
index_mrr :===>  2
mrr score :===>  0.5
Makanan & Minuman  Acc ==>  0.5714285714285714
index_mrr :===>  1
mrr score :===>  1.0
Makanan & Minuman  Acc ==>  0.2857142857142857
index_mrr :===>  1
mrr score :===>  1.0
Fashion & Bag  Acc ==>  0.35714285714285715
index_mrr :===>  6
mrr score :===>  0.16666666666666666
Merchandise  Acc ==>  0.21428571428571427
index_mrr :===>  1
mrr score :===>  1.0
Merchandise  Acc ==>  0.21428571428571427
index_mrr :===>  1
mrr score :===>  1.0
Merchandise  Acc ==>  0.21428571428571427
index_mrr :===>  1
mrr score :===>  1.0
Merchandise  Acc ==>  0.21428571428571427
index_mrr :===>  1
mrr score :===>  1.0
Fashion & Bag  Acc ==>  0.7857142857142857
index_mrr :===>  1
mrr score :===>  1.0
Fashion & Bag  Acc ==>  0.7142857142857143
index_mrr :===>  1
mrr score :===>  1.0
Fashion & Bag  Acc ==>  0.7142857142857143
index_mrr :===>

In [11]:
# Tampilkan dokumen yang paling mirip dengan dokumen pertama

data = {
    'True_Label': [],
    'Predicted_label': [],
    'Predicted Sub Category': [],
    'Nama Produk':[],
    'ID Produk': [],
    'Similarity': [],
    'Deskripsi':[],
}

limit = 15
n_training = 1500
acc=0

for i in range(n_training):
    # index_document = random.randint(1, 1800)

    index_document = i
    cosine = cosine_similarity_limit(data,limit,index_document,detail_output=False)
    acc+=cosine


result_df = pd.DataFrame(data)

# Menyimpan DataFrame ke dalam file CSV
result_df.to_csv('cosine_output.csv', index=False)

print("Akurasi rata-rata: ", acc/n_training)

Makanan & Minuman  ==>  0.7333333333333333
Makanan & Minuman  ==>  0.7333333333333333
Makanan & Minuman  ==>  1.0
Makanan & Minuman  ==>  1.0
Fashion & Bag  ==>  0.6
Merchandise  ==>  0.4
Merchandise  ==>  0.4
Merchandise  ==>  0.4
Merchandise  ==>  0.4
Fashion & Bag  ==>  0.8666666666666667
Fashion & Bag  ==>  0.7333333333333333
Fashion & Bag  ==>  0.6666666666666666
Fashion & Bag  ==>  0.6
Fashion & Bag  ==>  0.9333333333333333
Fashion & Bag  ==>  0.6666666666666666
Fashion & Bag  ==>  0.8
Fashion & Bag  ==>  0.4
Fashion & Bag  ==>  0.7333333333333333
Fashion & Bag  ==>  0.8
Fashion & Bag  ==>  0.8666666666666667
Fashion & Bag  ==>  0.8
Fashion & Bag  ==>  0.8
Fashion & Bag  ==>  0.8
Fashion & Bag  ==>  0.5333333333333333
Kain Tenun  ==>  0.6666666666666666
Kain Songket  ==>  0.3333333333333333
Kain Tenun  ==>  0.4666666666666667
Kain Tenun  ==>  0.8
Kain Songket  ==>  0.3333333333333333
Kain Songket  ==>  0.4
Kain Songket  ==>  0.3333333333333333
Kain Tenun  ==>  0.26666666666666666